<a href="https://colab.research.google.com/github/owoMarciN/Python/blob/main/Intro-AI/Lab-IV/AI_Breast_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn
!pip install torch

In [19]:
# SciKit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()

In [3]:
# Dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [4]:
# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim

In [5]:
# Ploting
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [22]:
X = data["data"]
y = data["target"]
feature_names = data["feature_names"]
target_names = data["target_names"]

df = pd.DataFrame(X, columns=feature_names)
df["target"] = y # 0 = malignant, 1 = benign (or vice versa depending on version)

print(df.head())
print("Features:", feature_names)
print("Targets:", target_names)

   mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
0        17.99         10.38          122.80     1001.0          0.11840   
1        20.57         17.77          132.90     1326.0          0.08474   
2        19.69         21.25          130.00     1203.0          0.10960   
3        11.42         20.38           77.58      386.1          0.14250   
4        20.29         14.34          135.10     1297.0          0.10030   

   mean compactness  mean concavity  mean concave points  mean symmetry  \
0           0.27760          0.3001              0.14710         0.2419   
1           0.07864          0.0869              0.07017         0.1812   
2           0.15990          0.1974              0.12790         0.2069   
3           0.28390          0.2414              0.10520         0.2597   
4           0.13280          0.1980              0.10430         0.1809   

   mean fractal dimension  ...  worst texture  worst perimeter  worst area  \
0             